Today preventing heart diseases has become more than necessary. Good data-driven systems for predicting heart diseases can improve the entire research, making sure that more people can live healthy lives. This is where Machine Learning comes into play. Machine Learning helps in predicting the Heart diseases, and the predictions made are quite accurate.

<b>Problem Description :</b>

A dataset is formed by taking into consideration some of the information of 916 people. The problem is to calculate whether that person will suffer from heart disease or not.

The dataset consists of 14 features and 916 samples with many <b>missing values</b>. The features used in here are:

   1. age: age of the person

   2. sex: 1 for male, 0 for female

   3. chest-pain type (cp): the type of chest pain they are experiencing. 1 for typical angina, 2 for atypical angina, 3 for 
   non-anginal pain and 4 for asymptomatic.

   4. resting blood pressure (trestbps)

   5. cholersterol(chol)

   6. fasting blood sugar(fbs): 0 (false) if the blood sugar is below 120 mg/dl, else 1 (true).

   7. resting electrocardiographic measurement (restecg): 0 for normal, 1 for having ST-T wave abnormality, 2 for  probable or definite left ventricular hyperthrophy.

   8. thalach: max heart rate achieved

   9. excercise induced angina(exang): 1 for yes, 0 for no.

   10. oldpeak: ST depression induced by exercise relative to rest.

   11. slope: the slope of the peak exercise ST segment. 1 for upsloping, 2 for flat and 3 for downsloping.

   12. peak exercise ST segment: 1 for upsloping, 2 for flat, 3 for downsloping.

   12. ca: number of major vessels (0-3) colored by flourosopy

   13. thal: displays the thalassemia: 3 for normal, 6 for fixed defect, and 7 for reversable defect.

   14. The last column shows that with the menstioned features, that person was diagnosed with heart disease or not. 0 shows the absence of disease and 1 shows the presence of disease.
   
<b>The target</b> is to diagnosis the heart disease, To see whether a person with some symptoms is suffering from the
disease or not.

In [1]:
# download the dataset from: https://www.kaggle.com/ronitf/heart-disease-uci
# then put it in a folder named "data".

import os
print(os.listdir())

['.ipynb_checkpoints', '.vscode', 'data', 'heart disease detection.ipynb', 'tree.dot', 'tree.png']


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns

from datetime import datetime
from scipy import *
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Reading the CSV file

df= pd.read_csv('./data/heart.csv')
df.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [ ]:
# Let's change the column names to be a bit clearer
df = df.rename(columns={'cp':'chest_pain_type','trestbps':'resting_blood_pressure','chol':'cholesterol','fbs':'fasting_blood_sugar',
'thalach':'max_heart_rate_achieved','exang':'exercise_induced_angina','ca':'no. of major vessels','thal':'thalassemia'})

#New show columns
df.columns

Index(['age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
       'cholesterol', 'fasting_blood_sugar', 'restecg',
       'max_heart_rate_achieved', 'exercise_induced_angina', 'oldpeak',
       'slope', 'no. of major vessels', 'thalassemia', 'target'],
      dtype='object')

In [ ]:
# To have a better understanding let's change the values from catergorical to non-categorical.
df['sex'][df['sex'] == 0] = 'female'
df['sex'][df['sex'] == 1] = 'male'

df['chest_pain_type'][df['chest_pain_type'] == 1] = 'typical angina'
df['chest_pain_type'][df['chest_pain_type'] == 2] = 'atypical angina'
df['chest_pain_type'][df['chest_pain_type'] == 3] = 'non-anginal pain'
df['chest_pain_type'][df['chest_pain_type'] == 4] = 'asymptomatic'

df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 0] = 'lower than 120mg/ml'
df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 1] = 'greater than 120mg/ml'

df['restecg'][df['restecg'] == 0] = 'normal'
df['restecg'][df['restecg'] == 1] = 'ST-T wave abnormality'
df['restecg'][df['restecg'] == 2] = 'left ventricular hypertrophy'

df['exercise_induced_angina'][df['exercise_induced_angina'] == 0] = 'no'
df['exercise_induced_angina'][df['exercise_induced_angina'] == 1] = 'yes'

df['slope'][df['slope'] == 1] = 'upsloping'
df['slope'][df['slope'] == 2] = 'flat'
df['slope'][df['slope'] == 3] = 'downsloping'

df['thalassemia'][df['thalassemia'] == 1] = 'normal'
df['thalassemia'][df['thalassemia'] == 2] = 'fixed defect'
df['thalassemia'][df['thalassemia'] == 3] = 'reversable defect'

In [ ]:
df.head(10)

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,restecg,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope,no. of major vessels,thalassemia,target
0,63,male,non-anginal pain,145,233,greater than 120mg/ml,normal,150,no,2.3,0,0,normal,1
1,37,male,atypical angina,130,250,lower than 120mg/ml,ST-T wave abnormality,187,no,3.5,0,0,fixed defect,1
2,41,female,typical angina,130,204,lower than 120mg/ml,normal,172,no,1.4,flat,0,fixed defect,1
3,56,male,typical angina,120,236,lower than 120mg/ml,ST-T wave abnormality,178,no,0.8,flat,0,fixed defect,1
4,57,female,0,120,354,lower than 120mg/ml,ST-T wave abnormality,163,yes,0.6,flat,0,fixed defect,1
5,57,male,0,140,192,lower than 120mg/ml,ST-T wave abnormality,148,no,0.4,upsloping,0,normal,1
6,56,female,typical angina,140,294,lower than 120mg/ml,normal,153,no,1.3,upsloping,0,fixed defect,1
7,44,male,typical angina,120,263,lower than 120mg/ml,ST-T wave abnormality,173,no,0.0,flat,0,reversable defect,1
8,52,male,atypical angina,172,199,greater than 120mg/ml,ST-T wave abnormality,162,no,0.5,flat,0,reversable defect,1
9,57,male,atypical angina,150,168,lower than 120mg/ml,ST-T wave abnormality,174,no,1.6,flat,0,fixed defect,1


In [ ]:
df.dtypes

age                          int64
sex                         object
chest_pain_type             object
resting_blood_pressure       int64
cholesterol                  int64
fasting_blood_sugar         object
restecg                     object
max_heart_rate_achieved      int64
exercise_induced_angina     object
oldpeak                    float64
slope                       object
no. of major vessels         int64
thalassemia                 object
target                       int64
dtype: object

In [ ]:
# Cast a pandas object to a specified dtype dtype.
# df['sex'] = df['sex'].astype('object')
# df['chest_pain_type'] = df['chest_pain_type'].astype('object')
# df['fasting_blood_sugar'] = df['fasting_blood_sugar'].astype('object')
# df['restecg'] = df['restecg'].astype('object')
# df['exercise_induced_angina'] = df['exercise_induced_angina'].astype('object')
# df['slope'] = df['slope'].astype('object')
# df['thalassemia'] = df['thalassemia'].astype('object')

In [ ]:
df.shape

(303, 14)

In [ ]:
#Now, let's check the missing data.
print('Data Sum of Null Values \n')
df.isnull().sum()

Data Sum of Null Values 



age                        0
sex                        0
chest_pain_type            0
resting_blood_pressure     0
cholesterol                0
fasting_blood_sugar        0
restecg                    0
max_heart_rate_achieved    0
exercise_induced_angina    0
oldpeak                    0
slope                      0
no. of major vessels       0
thalassemia                0
target                     0
dtype: int64

In [ ]:
# Since we don't need the first column

df = pd.get_dummies(df,drop_first=True) 
df.head(10)

,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,oldpeak,no. of major vessels,target,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,fasting_blood_sugar_lower than 120mg/ml,restecg_left ventricular hypertrophy,restecg_normal,exercise_induced_angina_yes,slope_flat,slope_upsloping,thalassemia_fixed defect,thalassemia_normal,thalassemia_reversable defect
0,63,145,233,150,2.3,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0
1,37,130,250,187,3.5,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0
2,41,130,204,172,1.4,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0
3,56,120,236,178,0.8,0,1,1,0,0,1,1,0,0,0,1,0,1,0,0
4,57,120,354,163,0.6,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0
5,57,140,192,148,0.4,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0
6,56,140,294,153,1.3,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0
7,44,120,263,173,0.0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,1
8,52,172,199,162,0.5,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1
9,57,150,168,174,1.6,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0


In [ ]:
#Split
x_train, x_test, y_train, y_test = train_test_split(df.drop('target', 1), df['target'], test_size = 0.2, random_state=10)

In [ ]:
# Train the data with Random Forest Classifier with 5 levels, counting from 0 from the root.
model = RandomForestClassifier(max_depth=5)
model.fit(x_train, y_train)

estimator = model.estimators_[1]
feature_names = [i for i in x_train.columns]

y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values

In [ ]:
#code from https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c

from sklearn.tree import export_graphviz

estimator = model.estimators_[1]
feature_names = [i for i in x_train.columns]

y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = y_train_str,
                rounded = True, proportion = True, 
                label='root',
                precision = 2, filled = True)

from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

from IPython.display import Image
Image(filename = 'tree.png')